# Data Science Essentials: Data Cleaning
    Matthew Mella
    10/31/23
    
    

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first, second, and third graders.
The NNAT has 48 questions. Each test assigns 1 point to each question asked (so there are no non integer scores).
Using this dataset, answer the following questions.



1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage). Make sure the second value is a percent.

2) List the columns that should be numeric that aren't. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.


In [80]:
# Read in data
df = pd.read_csv("g_t_results.csv")

# Highest number of null values
na_vals = df.isna().sum()
highest_na_col = na_vals.idxmax()
highest_na_percentage = (na_vals.max() / df.shape[0]) * 100  # Convert to percent
print((highest_na_col, f"{highest_na_percentage}%"))

# Columns that should be numeric
non_numeric_cols = df.select_dtypes(['object']).columns
should_be_numeric = [col for col in non_numeric_cols if df[col].str.isnumeric().any()]
print(tuple(should_be_numeric))

# Number of 3rd graders with invalid OLSAT Verbal Scores
third_graders = df[df['Entering Grade Level'] == '3']
invalid_scores = third_graders['OLSAT Verbal Score'].astype(float) > 30
print(invalid_scores.sum())

# Total missing data values
total_na_vals = na_vals.sum()
print(total_na_vals)

('School Assigned', '76.06837606837607%')
('Entering Grade Level', 'OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score', 'School Assigned')
1
193


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows by dropping the first **or** last. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [78]:

# read in the data
df = pd.read_csv("imdb.csv")

# drop duplicates
df.drop_duplicates(inplace=True)
print('Shape after dropping duplicates:', df.shape)

# drop rows with any missing values
df.dropna(inplace=True)
print('Shape after dropping rows with missing data:', df.shape)

# remove rows that do not meet the valid data ranges
df = df[(df['duration'] >= 30) & (df['imdb_score'] > 0) & (df['title_year'] > 2000)]

# drop columns with three or fewer unique values
cols_to_drop = df.columns[df.nunique() <= 3]
df.drop(cols_to_drop, axis=1, inplace=True)
print('Columns dropped:', tuple(cols_to_drop))

# convert the titles to all lower case
df['movie_title'] = df['movie_title'].str.lower()

# reset index
df.reset_index(drop=True, inplace=True)

# display the first five rows of the dataframe
print('First five rows of the cleaned dataframe:')

Shape after dropping duplicates: (93, 13)
Shape after dropping rows with missing data: (64, 13)
Columns dropped: ('color', 'language')
First five rows of the cleaned dataframe:


### Problem 3
Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt for your convenience.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    
    2) Add two new features: 
		a) Remodeled: Whether or not a house has been remodeled with a Y if it has been
		   remodeled, or a N if it has not.
		
		b) TotalPorch: Using the 5 different porch/deck columns, create a new column that
		   provides the total square footage of all the decks and porches for each house.
    
	3) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    4) Add a constant column to the dataframe.

    5) Save a copy of the dataframe.

	6) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	7) Run an OLS using all numerical data regression on your model.  

	
Print the ten features that have the highest coef in your model. Then print the summary. Don't print the OLS

In [71]:
# Load the data
df = pd.read_csv('housing.csv', index_col=0)

# Handle missing data
# For numerical features, fill missing values with the median
for column in df.select_dtypes(include=np.number).columns:
    df[column].fillna(df[column].median(), inplace=True)

# For categorical features, fill missing values with the 'missing'
for column in df.select_dtypes(include='object').columns:
    df[column].fillna('missing', inplace=True)

# New features
df['Remodeled'] = np.where(df['YearRemodAdd'] != 'missing', 'N', 'Y')
df['TotalPorch'] = df[['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].sum(axis=1)

# Identify misencoded variable and one-hot encode it
# Let's say 'MSSubClass' should be treated as categorical
df['MSSubClass'] = df['MSSubClass'].astype('object')
df = pd.get_dummies(df, columns=['MSSubClass'], drop_first=True)

# Add a constant column
df = sm.add_constant(df)

df_copy = df.copy()

# one-hot encode 4 categorical variables
df = pd.get_dummies(df, columns=['BldgType', 'HouseStyle', 'OverallQual', 'OverallCond'], drop_first=True)

# remove columns that are not numeric
df = df.select_dtypes(include=np.number)

results = sm.OLS(df['SalePrice'], df.drop(columns='SalePrice')).fit()

# sort and save the coefficients
coefs = results.params.sort_values(ascending=False)
print(coefs[:10])

print(results.summary())

GarageCars      14622.369705
BsmtFullBath     7439.832362
Fireplaces       7290.880558
TotRmsAbvGrd     7075.032041
FullBath         4816.844319
BsmtHalfBath     1111.564971
YearRemodAdd      450.599237
MoSold            372.404974
YearBuilt         315.893065
GarageYrBlt        75.540010
dtype: float64
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.775
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     159.0
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:39:57   Log-Likelihood:                -17454.
No. Observations:                1460   AIC:                         3.497e+04
Df Residuals:                    1428   BIC:                         3.514e+04
Df Model:                          31                          

### Problem 4

Using the copy of the dataframe you created in Problem 3, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [75]:
df = df_copy.copy()
# one-hot encode all categorical variables
df = pd.get_dummies(df, drop_first=True)

print('Shape of the Data Frame:', df.shape)
df = df.select_dtypes(include=np.number)

# fit the model
results = sm.OLS(df['SalePrice'], df.drop(columns='SalePrice')).fit()

# sort and save the coefficients
coefs = results.params.sort_values(ascending=False)
print(coefs[:10])

print(results.summary())

Shape of the Data Frame: (1460, 276)
const           148290.164471
OverallQual      16575.588828
GarageCars       10987.446448
BsmtFullBath      7919.348799
TotRmsAbvGrd      5880.316579
OverallCond       4922.673178
Fireplaces        3678.949927
FullBath          2657.277845
BsmtHalfBath       586.302755
YearBuilt          260.552427
dtype: float64
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     181.3
Date:                Tue, 07 Nov 2023   Prob (F-statistic):               0.00
Time:                        13:42:02   Log-Likelihood:                -17341.
No. Observations:                1460   AIC:                         3.475e+04
Df Residuals:                    1426   BIC:                         3.493e+04
Df Model:       

While the R^2 of